In [4]:
from load_data import *
import pandas as pd

# pd.describe_option()
#pandas column별 최대 출력 길이 조절
pd.set_option("max_colwidth",200)
pd.set_option('display.max_rows', 50)

In [5]:
#train.csv 불러오기, 중복 및 오라벨링 데이터 정리

data = pd.read_csv("/opt/ml/dataset/train/train.csv")
# 완전 중복 제거 42개
data = data.drop_duplicates(['sentence', 'subject_entity', 'object_entity', 'label'], keep='first')
# 라벨링이 다른 데이터 제거
data = data.drop(index=[6749, 8364, 22258, 277, 25094])
data = data.reset_index(drop=True)

data['subject_entity'] = data.subject_entity.map(eval)
data['object_entity'] = data.object_entity.map(eval)

In [6]:
#라벨 종류
label_list = list(data.label.value_counts().index)
print(label_list)

['no_relation', 'org:top_members/employees', 'per:employee_of', 'per:title', 'org:member_of', 'org:alternate_names', 'per:origin', 'org:place_of_headquarters', 'per:date_of_birth', 'per:alternate_names', 'per:spouse', 'per:colleagues', 'per:parents', 'org:founded', 'org:members', 'per:date_of_death', 'org:product', 'per:children', 'per:place_of_residence', 'per:other_family', 'per:place_of_birth', 'org:founded_by', 'per:product', 'per:siblings', 'org:political/religious_affiliation', 'per:religion', 'per:schools_attended', 'org:dissolved', 'org:number_of_employees/members', 'per:place_of_death']


In [7]:
#subject와 object type 정리
def sbj_obj_info(row):
    return row["subject_entity"]["type"] + "_" + row["object_entity"]["type"]

def sbj_obj_len(row):
    return row["subject_entity"]["word"] + "_" + row["object_entity"]["word"]

data["sbj_obj_type"] = data.apply(sbj_obj_info,axis=1)
data["sbj_obj_word"] = data.apply(sbj_obj_len,axis=1)
print("Done")

Done


In [8]:
data.head(2)

,id,sentence,subject_entity,object_entity,label,source,sbj_obj_type,sbj_obj_word
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}",no_relation,wikipedia,ORG_PER,비틀즈_조지 해리슨
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': 23, 'type': 'ORG'}","{'word': '대안신당', 'start_idx': 14, 'end_idx': 17, 'type': 'ORG'}",no_relation,wikitree,ORG_ORG,민주평화당_대안신당


In [12]:
#라벨별 subject_object 빈도수 확인
tmp = data.groupby(["label","sbj_obj_type"]).sbj_obj_type.count().to_frame()
tmp.columns = ["freq"]
tmp = tmp.reset_index()

#label별 type-freq
c = 1
from collections import defaultdict
label_type_freq = defaultdict(dict)
for i in range(len(tmp)):
    row = tmp.iloc[i]
    label,sbj_obj_type, freq = row['label'],row['sbj_obj_type'], row["freq"]
    label_type_freq[label][sbj_obj_type] = freq
    # if freq ==1:
        # print(f"[{c}]{label}_{sbj_obj_type} : {freq}")
        # c+=1
print(f"전체 label_sbj_obj_type {len(tmp)}개 중 {c}개의 조건에서 데이터가 1개씩만 존재")


전체 label_sbj_obj_type 140개 중 1개의 조건에서 데이터가 1개씩만 존재


In [13]:
for key in label_type_freq.keys():
    print(f"{key} : {label_type_freq[key]}")

no_relation : {'ORG_DAT': 1582, 'ORG_LOC': 548, 'ORG_NOH': 192, 'ORG_ORG': 1954, 'ORG_PER': 396, 'ORG_POH': 723, 'PER_DAT': 505, 'PER_LOC': 309, 'PER_NOH': 118, 'PER_ORG': 739, 'PER_PER': 1396, 'PER_POH': 1051}
org:alternate_names : {'ORG_DAT': 1, 'ORG_LOC': 23, 'ORG_NOH': 5, 'ORG_ORG': 1153, 'ORG_PER': 31, 'ORG_POH': 105}
org:dissolved : {'ORG_DAT': 66}
org:founded : {'ORG_DAT': 450}
org:founded_by : {'ORG_ORG': 5, 'ORG_PER': 144, 'ORG_POH': 5, 'PER_LOC': 1}
org:member_of : {'ORG_DAT': 5, 'ORG_LOC': 173, 'ORG_NOH': 9, 'ORG_ORG': 1318, 'ORG_PER': 1, 'ORG_POH': 358}
org:members : {'ORG_DAT': 1, 'ORG_LOC': 97, 'ORG_NOH': 2, 'ORG_ORG': 285, 'ORG_PER': 2, 'ORG_POH': 32, 'PER_PER': 1}
org:number_of_employees/members : {'ORG_NOH': 48}
org:place_of_headquarters : {'ORG_DAT': 4, 'ORG_LOC': 893, 'ORG_NOH': 2, 'ORG_ORG': 254, 'ORG_PER': 1, 'ORG_POH': 39, 'PER_LOC': 1}
org:political/religious_affiliation : {'ORG_DAT': 1, 'ORG_LOC': 4, 'ORG_ORG': 54, 'ORG_PER': 1, 'ORG_POH': 38}
org:product : {'OR

In [111]:
#label - sub_obj - dictionary 만들기
from collections import defaultdict
sbj_obj_dict = defaultdict(dict)

for label in label_list:
    sbj_obj_type_list = data[data.label==label].sbj_obj_type.unique()
    for sbj_obj_type in sbj_obj_type_list:
        sbj_obj_dict[label][sbj_obj_type] = {}
        sbj_obj_dict[label][sbj_obj_type]["sbjs"] = []
        sbj_obj_dict[label][sbj_obj_type]["objs"] = []

In [112]:
from tqdm import tqdm
for i in tqdm(range(len(data))):
    tmp_row = data.iloc[i]
    label, sbj_obj_type = tmp_row["label"],tmp_row["sbj_obj_type"]
    sbj,obj = tmp_row["sbj_obj_word"].split("_")
    
    sbj_obj_dict[label][sbj_obj_type]["sbjs"].append(sbj)
    sbj_obj_dict[label][sbj_obj_type]["objs"].append(obj)

100%|██████████| 32423/32423 [00:03<00:00, 9283.37it/s]


In [113]:
label = "per:title"
sbj_obj_type = "PER_NOH"
len(sbj_obj_dict[label][sbj_obj_type]["sbjs"])

19

In [114]:
sbj_list = sbj_obj_dict[label][sbj_obj_type]["sbjs"]
sbj_list_or = sbj_list.copy()

In [139]:
random.seed(2)
[random.choice(sbj_list_or) for _ in range(len(sbj_list_or))]

['홍영표',
 '함태영',
 '함태영',
 '이학재',
 '김홍일',
 '부좌현',
 '유진산',
 '변인선',
 '홍영표',
 '김서형',
 '김홍일',
 '김대중',
 '정세균',
 '이석기',
 '이학재',
 '송석찬',
 '이낙연',
 '이석기',
 '유진산']

In [116]:
i=0
while i<1000 or sbj_list!=sbj_list_or:
    random.seed(2)
    sbj_list = [random.choice(sbj_list_or) for _ in range(len(sbj_list_or))]
# sbj_list_or = sbj_list.copy()
    # random.shuffle(sbj_list)

    if sbj_list==sbj_list_or:
        print(i)
        break
    i+=1
print(i)
# sbj_list==sbj_list_or

# [random.choice(sbj_list) for _ in range(50)]

KeyboardInterrupt: 

In [17]:
label = "per:title"
sbj_obj_type = "PER_NOH"
sbj_list = sbj_obj_dict[label][sbj_obj_type]["sbjs"]
obj_list = sbj_obj_dict[label][sbj_obj_type]["objs"]
# resample=True
# while resample==True:
    # subject와 object entity 순서 shuffle
    # random.seed(2)
    # random.shuffle(sbj_list)
    # random.shuffle(obj_list)

    # #shuffle된 entity를 조합
    # new_sb_ob_list = []
    # for sb,ob in zip(sbj_list,obj_list):
    #     new_sb_ob_list.append(f"{sb}_{ob}")
    
    # #기존 entity 조합과 비교
    # data_cond = (data.label==label) & (data.sbj_obj_type==sbj_obj_type)
    
    # if not any(data[data_cond].sbj_obj_word==new_sb_ob_list): #전체가 false(모든 element 불일치)일 경우 return
    #     resample=False

In [ ]:
target_data = data[data_cond]
target_data["replaced"] = new_sb_ob_list
target_data.head(2)

In [11]:
def check_entity_loc(row):
    row = row.copy()
    sent = row['sentence']
    sbjs, objs = row['subject_entity'], row['object_entity']
    new_sb, new_ob = row['replaced'].split("_")

    sb_len, new_sb_len, ob_len, new_ob_lens = len(sbjs['word']), len(new_sb), len(objs['word']), len(new_ob)
    if sbjs['start_idx'] < objs['start_idx'] : 
        str1,str2,str3 = sent[:sbjs['start_idx']], sent[sbjs['end_idx']+1:objs['start_idx']], sent[objs['end_idx']+1:]
        new_sent = str1 + new_sb + str2 + new_ob + str3
        sbjs['word'] = new_sb
        sbjs["end_idx"] = sbjs['start_idx'] + len(new_sb)-1
        objs['word'] = new_ob
        objs['start_idx'] = new_sent.find(new_ob)
        objs['end_idx'] = objs['start_idx'] + len(new_ob)-1
    else:
        str1,str2,str3 = sent[:objs['start_idx']], sent[objs['end_idx']+1:sbjs['start_idx']], sent[sbjs['end_idx']+1:]
        new_sent = str1 + new_ob + str2 + new_sb + str3
        objs['word'] = new_ob
        objs["end_idx"] = objs['start_idx'] + len(new_sb)-1
        sbjs['word'] = new_sb
        sbjs['start_idx'] = new_sent.find(new_sb)
        sbjs['end_idx'] = sbjs['start_idx'] + len(new_sb)-1
    
    return {"id" : row['id'],
            "sentence" : new_sent,
            "subject_entity" : sbjs,
            "object_entity" : objs,
            "label" : row["label"],
            "source" : row["source"],
            "sbj_obj_type" : row["sbj_obj_type"],
            "sbj_obj_word" : row["sbj_obj_word"]}

# id	sentence	subject_entity	object_entity	label	source	sbj_obj_type	sbj_obj_word	rget_data.head(2)
# target_data.iloc[:1]
ttt = target_data[:2].apply(check_entity_loc,axis=1)
# target_data.iloc[:2]

NameError: name 'target_data' is not defined

In [ ]:
pd.DataFrame(list(target_data.iloc[:2].apply(check_entity_loc,axis=1).values))

In [ ]:
ttt = pd.DataFrame(list(target_data.apply(check_entity_loc,axis=1).values))
ttt

In [ ]:
# pd.concat(ttt,data[:2])
pd.concat([data,pd.DataFrame(list(target_data.iloc[:2].apply(check_entity_loc,axis=1).values))])

In [ ]:
data

In [ ]:

pd_dataset = pd.read_csv('/opt/ml/dataset/train/train.csv')
    
# 완전 중복 제거 42개
pd_dataset = pd_dataset.drop_duplicates(['sentence', 'subject_entity', 'object_entity', 'label'], keep='first')
# 라벨링이 다른 데이터 제거
pd_dataset = pd_dataset.drop(index=[6749, 8364, 22258, 277, 25094])
pd_dataset = pd_dataset.reset_index(drop=True)
print("Finish remove duplicated data")
#datatype 변경
pd_dataset['subject_entity'] = pd_dataset.subject_entity.map(eval)
pd_dataset['object_entity'] = pd_dataset.object_entity.map(eval)


In [ ]:
import collections

In [151]:
def load_data(dataset_dir, entity_flag=0, preprocessing_flag=None, mecab_flag=0,seed=2):
    """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    
    
    # 완전 중복 제거 42개
    pd_dataset = pd_dataset.drop_duplicates(['sentence', 'subject_entity', 'object_entity', 'label'], keep='first')
    # 라벨링이 다른 데이터 제거
    pd_dataset = pd_dataset.drop(index=[6749, 8364, 22258, 277, 25094])
    pd_dataset = pd_dataset.reset_index(drop=True)
    # pd_dataset = pd_dataset.iloc
    print("Finish remove duplicated data")

    #datatype 변경
    pd_dataset['subject_entity'] = pd_dataset.subject_entity.map(eval)
    pd_dataset['object_entity'] = pd_dataset.object_entity.map(eval)
    
    #data augmentation(동일한 label 및 subject_object 내에서 subject와 object entity를 sampling하여 데이터 증량)
    #단 특정 subject_object_type은 data가 1개이므로, 이 경우 임의로 값 지정
    pd_dataset = augmentation_by_resampling(pd_dataset,seed)

    return pd_dataset


def augmentation_by_resampling(data,seed):
    #subject_object 타입 및 word를 새로운 column에 추가(마지막에 삭제 예정)
    def sbj_obj_type(row):
        return row["subject_entity"]["type"] + "_" + row["object_entity"]["type"]
    def sbj_obj_word(row):
        return row["subject_entity"]["word"] + "_" + row["object_entity"]["word"]

    data["sbj_obj_entity_type"] = data.apply(sbj_obj_type,axis=1)
    data["sbj_obj_entity_word"] = data.apply(sbj_obj_word,axis=1)
       

    #라벨 종류 확인
    label_list = get_labels(data)

    #label_sbj_obj - dict 생성
    sbj_obj_dict = get_dict_frame(data,label_list)

    #dictionary에 값 입력 - label - sbj_obj_entity_type - sbj / obj
    print("데이터 수집 시작")
    for i in range(len(data)):
        tmp_row = data.iloc[i]
        label, sbj_obj_entity_type = tmp_row["label"],tmp_row["sbj_obj_entity_type"]
        sbj,obj = tmp_row["sbj_obj_entity_word"].split("_")
        sbj_obj_dict[label][sbj_obj_entity_type]["sbjs"].append(sbj)
        sbj_obj_dict[label][sbj_obj_entity_type]["objs"].append(obj)
    
    print("데이터 샘플링 시작")
    #sampling
    print("샘플링 시작 시: ", len(data))
    for label in label_list:
        for sbj_obj_entity_type in sbj_obj_dict[label].keys():
            print(label,"_",sbj_obj_entity_type)
            if len(sbj_obj_dict[label][sbj_obj_entity_type]['sbjs'])>5: #data개수가 1개이면 shuffle이 불가하므로 pass
                new_sbj_obj = shuffling_data(data,sbj_obj_dict,label,sbj_obj_entity_type, seed)
                data = augmentation(data,new_sbj_obj,label,sbj_obj_entity_type)
                print("샘플링 중", len(data))
    # data = data.drop(['sbj_obj_entity_type','sbj_obj_entity_word'])
    return data

             

def get_labels(data):
    return list(data.label.value_counts().index)


def get_dict_frame(data,label_list):
    sbj_obj_dict = collections.defaultdict(dict)
    for label in label_list:
        sbj_obj_type_list = data.loc[data.label==label,"sbj_obj_entity_type"].unique()
        for sbj_obj_type in sbj_obj_type_list:
            sbj_obj_dict[label][sbj_obj_type] = {}
            sbj_obj_dict[label][sbj_obj_type]["sbjs"] = []
            sbj_obj_dict[label][sbj_obj_type]["objs"] = []
    return sbj_obj_dict


def shuffling_data(data,sbj_obj_dict,label,sbj_obj_type, seed=2):
    sbj_list = sbj_obj_dict[label][sbj_obj_type]["sbjs"]
    obj_list = sbj_obj_dict[label][sbj_obj_type]["objs"]
    resample=True
    while resample==True:
        #subject와 object entity 순서 shuffle
        random.seed(seed)
        random.shuffle(sbj_list)
        random.shuffle(obj_list)
        sbj_list_choice = [random.choice(sbj_list) for _ in range(len(sbj_list))]
        obj_list_choice = [random.choice(obj_list) for _ in range(len(obj_list))]

        #shuffle된 entity를 조합
        new_sb_ob_list = []
        for sb,ob in zip(sbj_list_choice,obj_list_choice):
            new_sb_ob_list.append(f"{sb}_{ob}")
        
        #기존 entity 조합과 비교
        data_cond = (data["label"]==label) & (data["sbj_obj_entity_type"]==sbj_obj_type)
        
        if not any(data.loc[data_cond,"sbj_obj_entity_word"]==new_sb_ob_list): #전체가 false(모든 element 불일치)일 경우 return
            return new_sb_ob_list

def augmentation(data,new_sb_ob_list, label,sbj_obj_type):
    data_cond = (data.label==label) & (data.sbj_obj_entity_type==sbj_obj_type)
    target_data = data[data_cond].copy()
    target_data['replaced'] = new_sb_ob_list
    new_data = pd.DataFrame(list(target_data.apply(change_entity,axis=1).values))
    data = pd.concat([data, new_data])
    return data

def change_entity(row):
    row = row.copy()
    sent = row['sentence']
    sbjs, objs = row['subject_entity'], row['object_entity']
    new_sb, new_ob = row['replaced'].split("_")

    sb_len, new_sb_len, ob_len, new_ob_lens = len(sbjs['word']), len(new_sb), len(objs['word']), len(new_ob)
    if sbjs['start_idx'] < objs['start_idx'] : 
        str1,str2,str3 = sent[:sbjs['start_idx']], sent[sbjs['end_idx']+1:objs['start_idx']], sent[objs['end_idx']+1:]
        new_sent = str1 + new_sb + str2 + new_ob + str3
        sbjs['word'] = new_sb
        sbjs["end_idx"] = sbjs['start_idx'] + len(new_sb)-1
        objs['word'] = new_ob
        objs['start_idx'] = new_sent.find(new_ob)
        objs['end_idx'] = objs['start_idx'] + len(new_ob)-1
    else:
        str1,str2,str3 = sent[:objs['start_idx']], sent[objs['end_idx']+1:sbjs['start_idx']], sent[sbjs['end_idx']+1:]
        new_sent = str1 + new_ob + str2 + new_sb + str3
        objs['word'] = new_ob
        objs["end_idx"] = objs['start_idx'] + len(new_sb)-1
        sbjs['word'] = new_sb
        sbjs['start_idx'] = new_sent.find(new_sb)
        sbjs['end_idx'] = sbjs['start_idx'] + len(new_sb)-1
    
    return {"id" : row['id'],
            "sentence" : new_sent,
            "subject_entity" : sbjs,
            "object_entity" : objs,
            "label" : row["label"],
            "source" : row["source"],
            "sbj_obj_entity_type" : row["sbj_obj_entity_type"],
            "sbj_obj_entity_word" : row["sbj_obj_entity_type"]}



In [22]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
dataset_dir = '/opt/ml/dataset/train/train.csv'
pd_dataset = pd.read_csv(dataset_dir)
    
if 'train' in dataset_dir:
    # 완전 중복 제거 42개
    pd_dataset = pd_dataset.drop_duplicates(['sentence', 'subject_entity', 'object_entity', 'label'], keep='first')
    # 라벨링이 다른 데이터 제거
    pd_dataset = pd_dataset.drop(index=[6749, 8364, 22258, 277, 25094])
    pd_dataset = pd_dataset.reset_index(drop=True)
    print("Finish remove duplicated data")

#datatype 변경
pd_dataset['subject_entity'] = pd_dataset.subject_entity.map(eval)
pd_dataset['object_entity'] = pd_dataset.object_entity.map(eval)



Finish remove duplicated data


In [23]:
X = pd_dataset[["id"]]#,"sentence",'subject_entity',"object_entity","source"]]
y = pd_dataset["label"]

In [25]:
y

0                      no_relation
1                      no_relation
2                    org:member_of
3        org:top_members/employees
4                      no_relation
                   ...            
32418              per:employee_of
32419               per:colleagues
32420    org:top_members/employees
32421                  no_relation
32422    org:top_members/employees
Name: label, Length: 32423, dtype: object

In [20]:
skf = StratifiedKFold(n_splits=5, shuffle=True,random_state=2)
Xs = []
ys = []
for i,(train_idx,valid_idx) in enumerate(skf.split(X,y)):
    train_dataset, valid_dataset = pd_dataset.iloc[train_idx], pd_dataset.iloc[valid_idx]
    break

